In [7]:
"""
MODIFICADO DE cpv2010arg.py en https://github.com/abenassi/pyredatam/blob/1480c481feb0698d54b59c3c17e52661a8c793df/pyredatam/cpv2010arg.py
"""

import pandas as pd
import numpy as np
import os
import warnings

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import bs4
from bs4 import BeautifulSoup
import requests

import time
import import_ipynb
from Redapy_query import query_final ## el código esta en el notebook llamado Redapy_query
import redapy
import datetime
begin_time = datetime.datetime.now()

# area = ["Distrito"]
# var1 = ["Poblacio.PAREA"]
# var2 = ["Poblacio.C5P82"]
# selection = ["Provinci 1501"]
# filter_a = "Distrito"

# query = query_final(tipo="frequency",variables1=var1,area_break=area)
#query = query_final(tipo="frequency",variables1=var1,area_break=area)

ser = Service(r'C:\\Users\\Dieguinchi\\Desktop\\chromedriver.exe')
# BASE_URL = "https://censos2017.inei.gob.pe/bininei/RpWebStats.exe/CmdSet?BASE=CPV2017DI&ITEM=PROGRED&lang=esp"

def make_query(query, url): # hace consulta "query" a redatam a través de procesador estadístico online
    print('Scrapeo iniciado')
    options = webdriver.ChromeOptions()#carga configuración del webdriver
    options.headless = True
    driver = webdriver.Chrome(options=options, service=ser)# el driver debe estar en archivos del pc. puede descargarse de https://chromedriver.chromium.org/downloads
    try: driver.get(url) # abre pagina web de redatam
    except: print('No se pudo abrir páginade REDATAM')
    print('Se cargó página REDATAM con éxito')
    query_input = driver.find_element(By.TAG_NAME,"textarea")# ubica linea de comandos
#     query_input.send_keys(query.decode("utf-8", "ignore"))
    query_input.send_keys(query) # escribe consulta en línea de comandos
    submit = driver.find_element(By.NAME,"Submit") #Busca botón "Ejecutar"
    submit.click()# clickea en "Ejecutar" y ejecuta consulta
    try: 
        WebDriverWait(driver, 3).until(lambda driver: len(driver.find_elements(By.XPATH,"//h2[contains(text(),'500 - Internal server error.')]")) == 1)
        print('No cargó la tabla. Error 505')
        return ""
    except: 
        print('Iniciando scrapeo...')
        WebDriverWait(driver, 999999).until(lambda driver: len(driver.find_elements(By.XPATH,"//*[contains(text(),'Descargar en formato Excel')]")) == 1)#espera 999 segundos o a que se muestren todas las tablas solicitadas, es decir, debe mostrar Descargar en formato Excel la misma cantidad de veces que la cantidad de variables de la solicitud 
        print('La tabla cargó correctamente')
        html = driver.find_element(By.ID,"tab-output")# obtiene unicamente la tabla de resultados
        html = html.get_attribute('outerHTML')# obtiene el html de la tabla de resultados
        driver.close() # cierra navegador
        return html # devuelve html con tabla de resultados

def make_dataframe(html): # crea dataframe con resultados
    try: 
        temp_table = pd.read_html(html) # lee todos los dataframes de la tabla de resultados
        print('Tabla scrapeada con éxito en:')
        tiempo=datetime.datetime.now() - begin_time
        print(tiempo)
        return temp_table
    except: 
        print('No se logró scrapear la tabla')
     # devuelve lista con dataframes

# concatena dataframes de la tabla de resultados en un solo dataframe
# merged = pd.concat(make_dataframe(make_query(query)))

# Excelwriter = pd.ExcelWriter("tablas.xlsx",engine="xlsxwriter") #exporta en excel dataframe final
# merged.to_excel(Excelwriter)
# Excelwriter.save()
# print(datetime.datetime.now() - begin_time)

In [10]:
# area = ["Departam"]
# var1 = ["Poblacio.P03ESC"]
# var2 = ["Poblacio.C5P20CODA"]
# selection = ["Provinci 1501"]
# filter_a = "Distrito"

# # query = query_final(tipo="frequency",variables1=var1,area_break=area)
# query = query_final(tipo="Frequency",variables1=var1,area_break=area)

# BASE_URL = "https://censos2017.inei.gob.pe/bininei/RpWebStats.exe/CmdSet?BASE=CPV2017DI&ITEM=PROGRED&lang=esp"

# merged = pd.concat(make_dataframe(make_query(query,BASE_URL)))

# # df = redapy.tabla_cruzada(merged)
# df = redapy.conversion_redatam_matriz(merged)

# Excelwriter = pd.ExcelWriter("tablas.xlsx",engine="xlsxwriter") #exporta en excel dataframe final
# df.to_excel(Excelwriter)
# Excelwriter.save()

Scrapeo iniciado
Se cargó página REDATAM con éxito
Iniciando scrapeo...
La tabla cargó correctamente
Tabla scrapeada con éxito en:
0:01:57.081761


C:\Users\Dieguinchi\AppData\Local\Temp\ipykernel_19692\150765746.py:19: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  Excelwriter.save()


In [ ]:
# import selenium
# print(selenium.__version__)